In [1]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='100GB')
client

/Users/bkirtman/miniconda3/envs/myenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59638 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59638/status,
Dashboard: http://127.0.0.1:59638/status,Workers: 5
Total threads: 20,Total memory: 465.66 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59639,Workers: 5
Dashboard: http://127.0.0.1:59638/status,Total threads: 20
Started: Just now,Total memory: 465.66 GiB
Comm: tcp://127.0.0.1:59660,Total threads: 4
Dashboard: http://127.0.0.1:59663/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:59646,


In [2]:
import numpy as np
import torch
import subprocess
import sys
import torch_harmonics as th
import torch_harmonics.distributed as dist
import pandas as pd
import matplotlib.pyplot as plt
from subs1_utils import *

In [3]:
# For MacOS is higher than 12.3+
if torch.backends.mps.is_available():
    print("Running on GPU")
#    print("Running on CPU")
    device = torch.device("mps")
#    device = torch.device("cpu")
    print("MPS is activated:",torch.backends.mps.is_built()) # Was the current version of PyTorch built with MPS activated?
else:
    print("Running  on CPU")
    device = torch.device("cpu")

device

Running on GPU
MPS is activated: True


device(type='mps')

In [4]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 63 # zonal wave number
mw = 63 # meridional wave number
kmax = 26
imax = 192
jmax = 96
steps_per_day = 216*1.5 ### Changing this number impliws time step changes and should
#                       be implemented carefully
###
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'TestNonLinearT63L26_Dist'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
toffset = 0
#
# If restart see below settings
#
#toffset = 1800*1 # toffset is the number of days that have already run
#datapath_init = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/Restarts/Total/' # Location in restarts
datapath_init = datapath #set equal to datapath if restarting in same directory
#
if ( toffset == 0): # Cold Start
    subprocess.call(['rm','-r', datapath])
    subprocess.check_output(['mkdir', datapath])

In [5]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg, lats = precompute_latitudes(jmax)
lats = 90-180*lats/(np.pi)
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Instantiate grid to spectral (dsht) and spectral to grid (disht) distibuted transforms
#
vsht = th.RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dsht = dist.DistributedRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
disht = dist.DistributedInverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
dvsht = dist.DistributedRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
divsht = dist.DistributedInverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [6]:
#
#
#
# Initialize spectral fields (at rest or to be read in)
def initialize(temp_newton,lnpsclim,kmax,mw,zw):
    for k in range (kmax):
        tmn1[k] = tmn1[k] + temp_newton[k]/1000.0
        tmn2[k] = tmn2[k] + temp_newton[k]/1000.0
        tmn3[k] = tmn3[k] + temp_newton[k]/1000.0
    qmn1 = lnpsclim/1000.0
    qmn2 = lnpsclim/1000.0
    qmn3 = lnpsclim/1000.0
    return tmn1,tmn2,tmn3,qmn1,qmn2,qmn3

In [7]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook preprocess
#              
#
# Implement at rest initial condition, but need coriolis since
# model predicts total vorticity
#
coriolis = np.broadcast_to([(4.0*np.pi/86400)*np.sin(lats[j,np.newaxis]*np.pi/180.0) for j in range(jmax)], (jmax, imax))
#
#
# Initialize spectral fields (at rest or to be read in)
# Need to read in background temperature data for Newtonian
# Relaxation and possible initialization, see preprocess
# for how to change source data or formulation.
#
temp_newton = torch.load('temp.spectral.pt') # newtonian 
#                                    relaxation temperature, spectral
lnpsclim = torch.load('lnps.spectral.pt') # lnps climatology for damping
#
# Read Climatology on Gausian Grid
#
tclim_gg = torch.load('tsig.ggrid.pt')
vortclim_gg = torch.load('vortsig.ggrid.pt')
divclim_gg = torch.load('divsig.ggrid.pt')
uclim_gg = torch.load('usig.ggrid.pt')
vclim_gg = torch.load('vsig.ggrid.pt')
dxqclim_gg = torch.load('dxq_gg.ggrid.pt')
dyqclim_gg = torch.load('dyq_gg.ggrid.pt')
#
divclim = dsht(divclim_gg)
vortclim = dsht(vortclim_gg)
tclim = dsht(tclim_gg)
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    zmn1 = torch.load(datapath_init+'zmn1.spectral.pt')
    zmn2 = torch.load(datapath_init+'zmn2.spectral.pt')
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128) 
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    dmn1 = torch.load(datapath_init+'dmn1.spectral.pt')
    dmn2 = torch.load(datapath_init+'dmn2.spectral.pt')
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    tmn1 = torch.load(datapath_init+'tmn1.spectral.pt') 
    tmn2 = torch.load(datapath_init+'tmn2.spectral.pt')
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
if ( toffset > 0 ): # Restart
    qmn1 = torch.load(datapath_init+'qmn1.spectral.pt')
    qmn2 = torch.load(datapath_init+'qmn2.spectral.pt')
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
if ( toffset == 0 ): # Only do this if cold start
    tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 =\
    initialize(tclim,lnpsclim,kmax,mw,zw)
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see preprocess for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
#
#
# Adding heating here see preprocess.ipynb
#
####heat = torch.load('heat_linear.ggrid.pt')
#
#
# or set to zero
#
heat = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
#

In [8]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [9]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = tmnt.detach().clone()
dmnt = tmnt.detach().clone()
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = tmnt.detach().clone()
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = ddtdiv.detach().clone()
ttend = ddtdiv.detach().clone()
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = ddtdiv .detach().clone()
temp = ddtdiv.detach().clone()
qdot = ddtdiv.detach().clone()
#
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 60
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    zmnt[daycount] = zmnt[daycount] + zmn1/steps_per_day
    dmnt[daycount] = dmnt[daycount] + dmn1/steps_per_day
    tmnt[daycount] = tmnt[daycount] + tmn1/steps_per_day
    qmnt[daycount] = qmnt[daycount] + qmn1/steps_per_day
    wmnt[daycount] = wmnt[daycount] + wmn1/steps_per_day
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        print((dmn1[10,2,2],dmn1[9,2,1]))
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl+toffset:total_days+toffset]
            postprocessing(disht,divsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
            wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            daycount = 0
        savedat = 0
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    vort = disht(zmn2) ### This is the relative vorticity
    div = disht(dmn2)
    temp = disht(tmn2)
    qdot = disht(wmn2)
    u,v = uv(divsht,zmn2,dmn2,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(divsht,qmn2,mw,zw,imax,jmax)
    #
    # Stack grid variables climo first[0] perturbation second[1]
    #
    vort2 = torch.stack((vortclim_gg,vort)) # vortclim is the absolute vorticity
    div2 = torch.stack((divclim_gg,div))
    temp2 = torch.stack((tclim_gg,temp))
    u2 = torch.stack((uclim_gg,u))
    v2 = torch.stack((vclim_gg,v))
    dxq2 = torch.stack((dxqclim_gg,dxq))
    dyq2 = torch.stack((dyqclim_gg,dyq))
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod_prescribed_mean(u2,v2,vort2,div2,temp2,dxq2,dyq2,heat,coriolis,delsig,si,sikap,slkap,\
                                                                r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv - lap_sht(dsht,e,mw,zw)
    _,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    #
    tmn3 = -ttend + dsht(ri)
    wmn3 = dsht(wj) ### Prescribed heating converted to spectral
    qmn3 = -dsht(cbar[1]) ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp_prescribed_mean(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,kmax,mw,zw)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = zmn3.detach().clone()
    tmn3 = zmn3.detach().clone()
    wmn3 = zmn3.detach().clone()
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done

(tensor(2.7843e-09-7.8964e-10j, dtype=torch.complex128), tensor(-6.4397e-10-5.6973e-10j, dtype=torch.complex128))
['Day = ', 1]
(tensor(-1.0313e-09+1.9167e-09j, dtype=torch.complex128), tensor(7.4396e-10-2.5038e-09j, dtype=torch.complex128))
['Day = ', 2]
(tensor(4.3183e-10+1.4342e-10j, dtype=torch.complex128), tensor(8.0857e-10-6.2726e-10j, dtype=torch.complex128))
['Day = ', 3]
(tensor(-1.9172e-09+8.6275e-10j, dtype=torch.complex128), tensor(-3.1215e-10+1.8140e-09j, dtype=torch.complex128))
['Day = ', 4]
(tensor(1.7898e-10+2.5247e-10j, dtype=torch.complex128), tensor(-2.0848e-09-8.8840e-11j, dtype=torch.complex128))
['Day = ', 5]
(tensor(-1.4707e-09-1.2485e-09j, dtype=torch.complex128), tensor(-9.9099e-10-2.0013e-09j, dtype=torch.complex128))
['Day = ', 6]
(tensor(1.1369e-09-1.1273e-09j, dtype=torch.complex128), tensor(4.6295e-10-1.0480e-10j, dtype=torch.complex128))
['Day = ', 7]
(tensor(1.6226e-10-3.6064e-10j, dtype=torch.complex128), tensor(1.1649e-10+4.1897e-10j, dtype=torch.comp

(tensor(2.7857e-07+8.7166e-08j, dtype=torch.complex128), tensor(-2.6285e-08-5.3698e-08j, dtype=torch.complex128))
['Day = ', 65]
(tensor(2.5588e-07+4.6979e-08j, dtype=torch.complex128), tensor(-2.0319e-08-2.5128e-09j, dtype=torch.complex128))
['Day = ', 66]
(tensor(1.4905e-07+2.9026e-08j, dtype=torch.complex128), tensor(-2.6405e-08-3.5206e-08j, dtype=torch.complex128))
['Day = ', 67]
(tensor(8.3817e-08+7.5463e-08j, dtype=torch.complex128), tensor(2.4377e-08-3.0693e-08j, dtype=torch.complex128))
['Day = ', 68]
(tensor(1.9305e-08+4.6875e-08j, dtype=torch.complex128), tensor(4.3691e-08+2.0840e-08j, dtype=torch.complex128))
['Day = ', 69]
(tensor(-3.4525e-08-2.6179e-08j, dtype=torch.complex128), tensor(7.1497e-08-5.2860e-08j, dtype=torch.complex128))
['Day = ', 70]
(tensor(-9.4412e-08+5.4998e-08j, dtype=torch.complex128), tensor(9.1594e-08-5.8770e-08j, dtype=torch.complex128))
['Day = ', 71]
(tensor(-9.8026e-08+1.2807e-07j, dtype=torch.complex128), tensor(4.6956e-08-3.8497e-08j, dtype=torc

(tensor(-1.1842e-07+4.9666e-08j, dtype=torch.complex128), tensor(-3.4251e-08+2.8859e-08j, dtype=torch.complex128))
['Day = ', 129]
(tensor(-6.3738e-08-1.1805e-09j, dtype=torch.complex128), tensor(-2.3968e-08+1.4177e-08j, dtype=torch.complex128))
['Day = ', 130]
(tensor(-4.2220e-08-3.9657e-08j, dtype=torch.complex128), tensor(-3.5434e-08+6.8527e-08j, dtype=torch.complex128))
['Day = ', 131]
(tensor(-2.0930e-08+7.8811e-10j, dtype=torch.complex128), tensor(-4.2667e-08+5.1509e-08j, dtype=torch.complex128))
['Day = ', 132]
(tensor(1.4255e-09+6.2183e-08j, dtype=torch.complex128), tensor(-5.4368e-08+7.8251e-08j, dtype=torch.complex128))
['Day = ', 133]
(tensor(1.0256e-07+7.3506e-08j, dtype=torch.complex128), tensor(-7.5624e-08+7.6356e-08j, dtype=torch.complex128))
['Day = ', 134]
(tensor(1.2479e-07+7.8107e-09j, dtype=torch.complex128), tensor(-6.6847e-08+6.7265e-08j, dtype=torch.complex128))
['Day = ', 135]
(tensor(4.5985e-08-8.0939e-08j, dtype=torch.complex128), tensor(-6.8888e-08+8.7514e-08

(tensor(-1.0065e-08+5.6084e-08j, dtype=torch.complex128), tensor(7.6923e-08+6.3404e-08j, dtype=torch.complex128))
['Day = ', 192]
(tensor(-1.5867e-08+6.9657e-08j, dtype=torch.complex128), tensor(3.7253e-08+2.7442e-08j, dtype=torch.complex128))
['Day = ', 193]
(tensor(-2.9121e-08+6.8162e-08j, dtype=torch.complex128), tensor(-4.9747e-09+3.4476e-08j, dtype=torch.complex128))
['Day = ', 194]
(tensor(-7.3550e-08+7.3187e-08j, dtype=torch.complex128), tensor(5.2548e-08+5.7957e-08j, dtype=torch.complex128))
['Day = ', 195]
(tensor(-6.8890e-08+5.5193e-08j, dtype=torch.complex128), tensor(4.2645e-08+4.0521e-08j, dtype=torch.complex128))
['Day = ', 196]
(tensor(-3.7034e-08+2.1731e-08j, dtype=torch.complex128), tensor(7.9698e-09+3.0615e-08j, dtype=torch.complex128))
['Day = ', 197]
(tensor(-5.5570e-08-1.2839e-08j, dtype=torch.complex128), tensor(2.0842e-08+2.5724e-08j, dtype=torch.complex128))
['Day = ', 198]
(tensor(-5.7981e-08-2.3556e-08j, dtype=torch.complex128), tensor(4.0049e-08+3.4893e-08j, 

(tensor(1.0923e-07+3.2372e-07j, dtype=torch.complex128), tensor(8.6755e-08-6.9991e-08j, dtype=torch.complex128))
['Day = ', 256]
(tensor(1.7815e-07+2.5190e-07j, dtype=torch.complex128), tensor(5.4194e-08-9.4095e-08j, dtype=torch.complex128))
['Day = ', 257]
(tensor(1.1136e-07+1.7770e-07j, dtype=torch.complex128), tensor(5.6811e-08-8.3366e-08j, dtype=torch.complex128))
['Day = ', 258]
(tensor(6.8414e-08+1.4743e-07j, dtype=torch.complex128), tensor(9.6977e-09-7.1217e-08j, dtype=torch.complex128))
['Day = ', 259]
(tensor(3.9012e-08+1.4721e-07j, dtype=torch.complex128), tensor(-2.3459e-08-4.6116e-08j, dtype=torch.complex128))
['Day = ', 260]
(tensor(5.4846e-08+1.1596e-07j, dtype=torch.complex128), tensor(-2.3372e-08-1.0218e-07j, dtype=torch.complex128))
['Day = ', 261]
(tensor(1.1470e-08+9.4572e-08j, dtype=torch.complex128), tensor(-1.4329e-08-1.0888e-07j, dtype=torch.complex128))
['Day = ', 262]
(tensor(-3.5432e-08+1.0468e-07j, dtype=torch.complex128), tensor(-5.8302e-08-6.7245e-08j, dtyp

...

(tensor(1.5018e-08+1.8386e-07j, dtype=torch.complex128), tensor(-2.6880e-08+3.7999e-10j, dtype=torch.complex128))
['Day = ', 1521]
(tensor(1.2235e-08+1.8483e-07j, dtype=torch.complex128), tensor(-2.6647e-08+2.2698e-08j, dtype=torch.complex128))
['Day = ', 1522]
(tensor(6.4869e-08+2.4802e-07j, dtype=torch.complex128), tensor(-1.1090e-07-1.9463e-08j, dtype=torch.complex128))
['Day = ', 1523]
(tensor(1.5101e-07+1.8988e-07j, dtype=torch.complex128), tensor(-8.9293e-08-2.1868e-08j, dtype=torch.complex128))
['Day = ', 1524]
(tensor(1.1786e-07+1.0007e-07j, dtype=torch.complex128), tensor(-1.0236e-07+9.3690e-09j, dtype=torch.complex128))
['Day = ', 1525]
(tensor(1.0953e-07+5.4132e-08j, dtype=torch.complex128), tensor(-1.3221e-07+4.8981e-09j, dtype=torch.complex128))
['Day = ', 1526]
(tensor(1.1974e-07+7.4830e-08j, dtype=torch.complex128), tensor(-1.5041e-07-3.5422e-08j, dtype=torch.complex128))
['Day = ', 1527]
(tensor(1.3323e-07+5.2872e-08j, dtype=torch.complex128), tensor(-1.2954e-07-1.6686e

(tensor(-1.5593e-08+1.4268e-07j, dtype=torch.complex128), tensor(-1.3407e-09-3.0362e-08j, dtype=torch.complex128))
['Day = ', 1584]
(tensor(4.0075e-08+1.5028e-07j, dtype=torch.complex128), tensor(-3.1534e-08-4.4782e-08j, dtype=torch.complex128))
['Day = ', 1585]
(tensor(5.5391e-08+9.8853e-08j, dtype=torch.complex128), tensor(-3.4734e-08-4.3323e-08j, dtype=torch.complex128))
['Day = ', 1586]
(tensor(6.0716e-08+9.8789e-08j, dtype=torch.complex128), tensor(-3.6103e-08-2.1229e-08j, dtype=torch.complex128))
['Day = ', 1587]
(tensor(6.9563e-08+1.5612e-07j, dtype=torch.complex128), tensor(-2.4740e-08-6.4320e-09j, dtype=torch.complex128))
['Day = ', 1588]
(tensor(1.6220e-07+1.2316e-07j, dtype=torch.complex128), tensor(-2.3530e-08+7.8582e-09j, dtype=torch.complex128))
['Day = ', 1589]
(tensor(2.1033e-07+2.3018e-08j, dtype=torch.complex128), tensor(3.3089e-08+3.0622e-08j, dtype=torch.complex128))
['Day = ', 1590]
(tensor(1.5272e-07-7.8427e-08j, dtype=torch.complex128), tensor(3.0350e-08+5.9322e-

(tensor(-1.9786e-08+4.5898e-07j, dtype=torch.complex128), tensor(-5.7792e-08+2.1469e-09j, dtype=torch.complex128))
['Day = ', 1647]
(tensor(1.2271e-07+3.8201e-07j, dtype=torch.complex128), tensor(-5.9178e-08+5.6066e-09j, dtype=torch.complex128))
['Day = ', 1648]
(tensor(1.9871e-07+2.9235e-07j, dtype=torch.complex128), tensor(-8.5359e-08-6.6024e-08j, dtype=torch.complex128))
['Day = ', 1649]
(tensor(1.5796e-07+1.3700e-07j, dtype=torch.complex128), tensor(-6.8944e-08-3.9795e-08j, dtype=torch.complex128))
['Day = ', 1650]
(tensor(1.2034e-07+8.5828e-08j, dtype=torch.complex128), tensor(-4.6555e-08+1.7579e-09j, dtype=torch.complex128))
['Day = ', 1651]
(tensor(6.7426e-08+6.7215e-08j, dtype=torch.complex128), tensor(-9.6235e-08-3.3276e-08j, dtype=torch.complex128))
['Day = ', 1652]
(tensor(8.2976e-08+1.0233e-07j, dtype=torch.complex128), tensor(-1.0512e-07-2.8636e-08j, dtype=torch.complex128))
['Day = ', 1653]
(tensor(8.3590e-08+5.6779e-08j, dtype=torch.complex128), tensor(-5.8707e-08-4.3319

(tensor(-7.4185e-08+7.0282e-08j, dtype=torch.complex128), tensor(6.5615e-09-1.1216e-08j, dtype=torch.complex128))
['Day = ', 1710]
(tensor(-9.0921e-08+2.2388e-08j, dtype=torch.complex128), tensor(-7.7422e-08-3.2166e-08j, dtype=torch.complex128))
['Day = ', 1711]
(tensor(-1.2451e-07-6.2175e-09j, dtype=torch.complex128), tensor(-4.5294e-08+4.6201e-09j, dtype=torch.complex128))
['Day = ', 1712]
(tensor(-1.8650e-07+4.4688e-08j, dtype=torch.complex128), tensor(-1.7867e-08+1.7757e-08j, dtype=torch.complex128))
['Day = ', 1713]
(tensor(-2.2467e-07+6.3849e-08j, dtype=torch.complex128), tensor(-5.2366e-08-1.1448e-08j, dtype=torch.complex128))
['Day = ', 1714]
(tensor(-1.8611e-07+9.1601e-08j, dtype=torch.complex128), tensor(-1.0280e-08-5.3186e-08j, dtype=torch.complex128))
['Day = ', 1715]
(tensor(-9.4463e-08+5.6546e-08j, dtype=torch.complex128), tensor(-1.7000e-08-3.8664e-08j, dtype=torch.complex128))
['Day = ', 1716]
(tensor(-6.4902e-08+3.6743e-09j, dtype=torch.complex128), tensor(4.9596e-10-4

(tensor(7.9309e-08+6.1195e-08j, dtype=torch.complex128), tensor(-6.4554e-08+1.7513e-08j, dtype=torch.complex128))
['Day = ', 1773]
(tensor(6.9202e-09+4.9712e-08j, dtype=torch.complex128), tensor(-1.1101e-08+3.5162e-08j, dtype=torch.complex128))
['Day = ', 1774]
(tensor(-1.8814e-08+1.4529e-08j, dtype=torch.complex128), tensor(-6.4716e-08+2.2250e-08j, dtype=torch.complex128))
['Day = ', 1775]
(tensor(-8.6077e-08+4.0951e-08j, dtype=torch.complex128), tensor(-7.7556e-09+4.2252e-08j, dtype=torch.complex128))
['Day = ', 1776]
(tensor(-3.4478e-08+9.6812e-08j, dtype=torch.complex128), tensor(-2.0276e-08+2.8840e-08j, dtype=torch.complex128))
['Day = ', 1777]
(tensor(-2.0935e-09+9.0494e-08j, dtype=torch.complex128), tensor(-6.5942e-08+6.4327e-09j, dtype=torch.complex128))
['Day = ', 1778]
(tensor(1.2689e-09+7.4000e-08j, dtype=torch.complex128), tensor(-1.2531e-07+7.5751e-09j, dtype=torch.complex128))
['Day = ', 1779]
(tensor(-4.7609e-08+2.4925e-08j, dtype=torch.complex128), tensor(-7.7424e-08-6.

In [10]:
## Write spectral data for possible restart
##
torch.save(zmn1,datapath+'zmn1.spectral.pt')
torch.save(zmn2,datapath+'zmn2.spectral.pt')
torch.save(dmn1,datapath+'dmn1.spectral.pt')
torch.save(dmn2,datapath+'dmn2.spectral.pt')
torch.save(tmn1,datapath+'tmn1.spectral.pt')
torch.save(tmn2,datapath+'tmn2.spectral.pt')
torch.save(qmn1,datapath+'qmn1.spectral.pt')
torch.save(qmn2,datapath+'qmn2.spectral.pt')